In [415]:
import pandas as pd
import numpy as np
import datetime as dt

In [416]:
df = pd.read_csv('AggregatedData.csv')

In [417]:
df = df.drop(columns='Holiday Sequence')

In [418]:
df['ATM Name'].value_counts()

KK Nagar ATM          2371
Christ College ATM    2355
Big Street ATM        2354
Mount Road ATM        2256
Airport ATM           2253
Name: ATM Name, dtype: int64

In [419]:
def process_date(date):
    if '/' in date:
        return dt.datetime(day=int(date.split('/')[1]), month=int(date.split('/')[0]), year=int(date.split('/')[-1]))
    if '-' in date:
        return dt.datetime(day=int(date.split('-')[0]), month=int(date.split('-')[1]), year=int(date.split('-')[-1]))
    return date

In [420]:
df['Transaction Date'] = df['Transaction Date'].apply(process_date)

In [421]:
df['Weekday'].value_counts()

SUNDAY       2177
SATURDAY     1131
TUESDAY       989
WEDNESDAY     947
MONDAY        945
THURSDAY      903
FRIDAY        902
Saturday      518
Thursday      517
Wednesday     515
Monday        513
Tuesday       513
Friday        510
Sunday        509
Name: Weekday, dtype: int64

In [422]:
'''def process_weekday(day):
    day = day.lower()
    if day == 'monday':
        return 1
    if day == 'tuesday':
        return 2
    if day == 'wednesday':
        return 3
    if day == 'thursday':
        return 4
    if day == 'friday':
        return 5
    if day == 'saturday':
        return 6
    if day == 'sunday':
        return 7
    return np.nan'''

"def process_weekday(day):\n    day = day.lower()\n    if day == 'monday':\n        return 1\n    if day == 'tuesday':\n        return 2\n    if day == 'wednesday':\n        return 3\n    if day == 'thursday':\n        return 4\n    if day == 'friday':\n        return 5\n    if day == 'saturday':\n        return 6\n    if day == 'sunday':\n        return 7\n    return np.nan"

In [423]:
#df['Weekday'] = df['Weekday'].apply(process_weekday)

In [424]:
#df['Location'] = df['ATM Name'].apply(lambda name: name.replace('ATM', '').strip())

In [425]:
df['Festival Religion'].value_counts()

NH    10727
H       367
N       198
M       178
C       119
Name: Festival Religion, dtype: int64

In [426]:
#df['Holiday Sequence'].value_counts()

In [427]:
df['Working Day'] = (df['Working Day'] == 'W').astype('int')

In [428]:
index = pd.DataFrame(index=pd.MultiIndex.from_product([
    pd.date_range(start=df['Transaction Date'].min(), end=df['Transaction Date'].max(), freq='1d'),
    set(df['ATM Name'])
], names=['Transaction Date', 'ATM Name']))

In [429]:
df = df.set_index(['Transaction Date', 'ATM Name']).join(index, how='right')

In [430]:
#df.join(index, on='Transaction Date', how='right').sort_values('Transaction Date')

In [431]:
df.columns

Index(['No Of Withdrawals', 'No Of XYZ Card Withdrawals',
       'No Of Other Card Withdrawals', 'Total amount Withdrawn',
       'Amount withdrawn XYZ Card', 'Amount withdrawn Other Card', 'Weekday',
       'Festival Religion', 'Working Day'],
      dtype='object')

In [432]:
df['Weekday'] = list(pd.to_datetime(df.index.get_level_values(0)).weekday + 1)

In [433]:
for day in [6, 7]:
    df['Working Day'][(df.isna().any(axis=1))&(df['Weekday'] == day)] = 0

for day in [1, 2, 3, 4, 5]:
    df['Working Day'][(df.isna().any(axis=1))&(df['Weekday'] == day)] = 1

/Users/ivan/Documents/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/ivan/Documents/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [434]:
df['Festival Religion'][df.isna().any(axis=1)] = 'NH'

/Users/ivan/Documents/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [435]:
for atm in set(df.index.get_level_values(1)):
    df[df.index.get_level_values(1) == atm] = df[df.index.get_level_values(1) == atm].fillna(df[df.index.get_level_values(1) == atm].mean().to_dict())

In [441]:
df.reset_index().rename(columns={
    'Transaction Date': 'date',
    'ATM Name': 'atm_name',
    'No Of Withdrawals': 'withdrawals_total',
    'No Of XYZ Card Withdrawals': 'withdrawals_xyz',
    'No Of Other Card Withdrawals': 'withdrawals_other',
    'Total amount Withdrawn': 'amount_total',
    'Amount withdrawn XYZ Card': 'amount_xyz',
    'Amount withdrawn Other Card': 'amount_other',
    'Weekday': 'weekday',
    'Festival Religion': 'festival',
    'Working Day': 'working_day'
}).to_csv('./data_clean.csv', index=False)